In [1]:
! uv pip install langchain google-genai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.11.14 environment at: C:\Users\Aditya Kumar Singh\anaconda4\envs\llmenv1
Audited 6 packages in 42ms


In [2]:
from google import genai
import os

from dotenv import load_dotenv

load_dotenv()


os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [3]:
%pip install -qU langchain-community pymupdf

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf"
loader = PyMuPDFLoader(file_path)
documents = loader.load()
documents[12].page_content

'11\n2.14 STUDY ABROAD PROGRAMME (SAP)\nStudy Abroad Programme (SAP) is offered to students of all disciplines which is important for \nglobal exposure. The students visit any one country of their choice, complete pre-defined \ncredits units under the international faculty. The advantage of SAP is hands-on experience of \ngaining knowledge of foreign culture, industry, and economic dynamics. It provides the \nstudents with an opportunity to interact with foreign faculty and carry out focused projects \nunder their expert guidance. For further details please go to URL https://www.amity.edu/sap/\n2.15 THREE CONTINENT PROGRAMMES (3C PROGRAMMES)\nAmity is a pioneer in developing and initiating 3C programmes in India. In this program, \nstudents gain unique exposure through studying in 3 different continents namely Asia, Europe \nand America. While the initial Semesters are completed at Amity University, as part of the \nprogram students spend 3 Months in U.K. and 3 Months in US.\nStudents 

In [15]:
from langchain_text_splitters import NLTKTextSplitter

In [16]:
text_splitter = NLTKTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

In [17]:
text_chunks = text_splitter.split_documents(documents)
text_chunks

Created a chunk of size 286, which is longer than the specified 200
Created a chunk of size 573, which is longer than the specified 200
Created a chunk of size 352, which is longer than the specified 200
Created a chunk of size 250, which is longer than the specified 200
Created a chunk of size 286, which is longer than the specified 200
Created a chunk of size 286, which is longer than the specified 200
Created a chunk of size 240, which is longer than the specified 200
Created a chunk of size 216, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 252, which is longer than the specified 200
Created a chunk of size 276, which is longer than the specified 200
Created a chunk of size 205, which is longer than the specified 200
Created a chunk of size 222, which is longer than the specified 200
Created a chunk of size 230, which is longer than the specified 200
Created a chunk of size 245, which is longer tha

[Document(metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf', 'file_path': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf', 'total_pages': 69, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-09-09T07:00:06+00:00', 'trapped': '', 'modDate': 'D:20250909070006Z', 'creationDate': '', 'page': 0}, page_content='STUDENT\nHANDBOOK\n(For Guidance & Compliance by Students)\n2025-2026'),
 Document(metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf', 'file_path': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf', 'total_pages': 69, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-09-09T07:00:06+00:00', 'trapped': '', 'modDate': 'D:20250909070006Z', 'creationDate': '', 'page': 1}, page_content='CONTENTS\nS. No.\n\nDescription \n \n \n \n \n \nPage No.

In [21]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [23]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [26]:
### Convert the text to embeddings
texts=[doc.page_content for doc in text_chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(text_chunks,embeddings)

Generating embeddings for 623 texts...


Batches: 100%|██████████| 20/20 [00:06<00:00,  3.21it/s]


Generated embeddings with shape: (623, 384)
Adding 623 documents to vector store...
Successfully added 623 documents to vector store
Total documents in collection: 623


In [27]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [28]:
rag_retriever

In [29]:
rag_retriever.retrieve("Where is amity")

Retrieving documents for query: 'Where is amity'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 34.25it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_3b1d77e7_33',
  'content': 'Amity has produced great leaders, entrepreneurs, artists, engineers, technologists, scientists, \nwriters, and above all wonderful human beings, who are strengthening the brand of Amity \nthrough their untiring efforts and knowledge in all walks of life.',
  'metadata': {'creationDate': '',
   'trapped': '',
   'moddate': '2025-09-09T07:00:06+00:00',
   'doc_index': 33,
   'subject': '',
   'creationdate': '',
   'creator': '',
   'format': 'PDF 1.5',
   'file_path': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',
   'total_pages': 69,
   'content_length': 252,
   'page': 4,
   'producer': 'iLovePDF',
   'source': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',
   'title': '',
   'keywords': '',
   'author': '',
   'modDate': 'D:20250909070006Z'},
  'similarity_score': 0.38267654180526733,
  'distance': 0.6173234581947327,
  'rank': 1},
 {'id': 'doc_2f8b6450_28',
  'content': "Amity is the world's largest \nUniversity campus having Platinu

In [30]:
rag_retriever.retrieve("ACADEMIC CALENDAR")

Retrieving documents for query: 'ACADEMIC CALENDAR'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 32.01it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_31dbc0c3_119',
  'content': '13\n2.19 ACADEMIC CALENDAR\nEvery year Academic Activity commences with the planning of Block Academic Calendar.',
  'metadata': {'page': 14,
   'content_length': 110,
   'creationdate': '',
   'producer': 'iLovePDF',
   'file_path': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',
   'modDate': 'D:20250909070006Z',
   'total_pages': 69,
   'creationDate': '',
   'doc_index': 119,
   'trapped': '',
   'creator': '',
   'source': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',
   'subject': '',
   'moddate': '2025-09-09T07:00:06+00:00',
   'format': 'PDF 1.5',
   'keywords': '',
   'author': '',
   'title': ''},
  'similarity_score': 0.5222317576408386,
  'distance': 0.4777682423591614,
  'rank': 1},
 {'id': 'doc_86523d03_407',
  'content': '45\nAcademic Calendar: Different types of calendars are available on Amizone e.g.',
  'metadata': {'producer': 'iLovePDF',
   'moddate': '2025-09-09T07:00:06+00:00',
   'page': 46,
   'total_pages': 69,

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [35]:
class GeminiLLM:
    def __init__(self, model_name: str = "gemini-2.5-flash", api_key: str = None):
        """
        Initialize Gemini LLM
        
        Args:
            model_name: Gemini model name
            api_key: Google API key (or set GOOGLE_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GOOGLE_API_KEY")
        
        if not self.api_key:
            raise ValueError("Google API key is required. Set GOOGLE_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGoogleGenerativeAI(
            google_api_key=self.api_key,
            model=self.model_name,
            temperature=0.1,
            max_output_tokens=1024
        )
        
        print(f"Initialized Gemini LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"

In [43]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    gemini_llm = GeminiLLM(api_key=os.getenv("GEMINI_API_KEY"))
    print("Gemini LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GEMINI_API_KEY environment variable to use the LLM.")
    gemini_llm = None

Initialized Gemini LLM with model: gemini-2.5-flash
Gemini LLM initialized successfully!


In [37]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("What is ACADEMIC CALENDAR")

Retrieving documents for query: 'What is ACADEMIC CALENDAR'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 23.77it/s]

Generated embeddings with shape: (1, 384)
Retrieved 4 documents (after filtering)


[{'id': 'doc_31dbc0c3_119',
  'content': '13\n2.19 ACADEMIC CALENDAR\nEvery year Academic Activity commences with the planning of Block Academic Calendar.',
  'metadata': {'creationDate': '',
   'subject': '',
   'file_path': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',
   'producer': 'iLovePDF',
   'moddate': '2025-09-09T07:00:06+00:00',
   'trapped': '',
   'format': 'PDF 1.5',
   'source': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',
   'author': '',
   'keywords': '',
   'total_pages': 69,
   'creationdate': '',
   'doc_index': 119,
   'creator': '',
   'content_length': 110,
   'modDate': 'D:20250909070006Z',
   'title': '',
   'page': 14},
  'similarity_score': 0.4536902904510498,
  'distance': 0.5463097095489502,
  'rank': 1},
 {'id': 'doc_86523d03_407',
  'content': '45\nAcademic Calendar: Different types of calendars are available on Amizone e.g.',
  'metadata': {'content_length': 80,
   'trapped': '',
   'source': '../data/HandBook_b9c6eSTUDENT HANDBOOK 2025.pdf',

In [39]:
### Simple RAG pipeline with Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Gemini LLM (set your GOOGLE_API_KEY in environment)
google_api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    google_api_key=google_api_key,
    model="gemini-2.5-flash",
    temperature=0.1,
    max_output_tokens=1024
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    ## retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answer using Gemini LLM
    prompt = f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response = llm.invoke(prompt)
    return response.content

In [44]:
answer=rag_simple("STUDENT SUPPORT SYSTEM, During online exams",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'STUDENT SUPPORT SYSTEM, During online exams'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 49.27it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


During online examinations, students receive support through Institutional Examination Help Desks (IEHD), University Examination Help Desks (UEHD), and University IT Help Desks (UITHD) via digital platforms like MS Team. For technical issues, students should contact their PL and IEHD.


In [42]:
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("STUDENT SUPPORT SYSTEM, During online exams", top_k=10, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'STUDENT SUPPORT SYSTEM, During online exams'
Top K: 10, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 27.52it/s]

Generated embeddings with shape: (1, 384)
Retrieved 6 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
During Online 
examinations, students will be provided support through Institutional Examination Help 
Desks (IEHD), University Examination Help Desks (UEHD) and Universit

y IT Help Desks 
(UITHD) by connecting through various digital platforms such as MS Team etc.

This helps students, parents, faculty and staff to access data from anywhere and 
anytime on web/mobile for:
• 
Admission
• 
Scholarships
• 
Regulations and Guidelines
• 
Registration-cum-Enrollment Process / Re-registration
• 
Academics - programmes, programme structure, course curriculum, course choices, 
master session plan, timetable, attendance, assessment scheme, faculty feedback, 
recorded classes, block academic calendar, fee calendar, programme academic calendar.

Students may 
contact their PL and Institutional Examination Help Desk (IEHD), if they face any technical 
issues during online examinations.

21
3.10  STUDENT SUPPORT SYSTEM
Amity University Uttar Pradesh provides support to its students through all aspects of their 
university experience, from first contact to beyond graduation with the objective of holistic 
development of students, to help students become more self-awar